# Web Scraping Homework - Mission to Mars

## Step 1 - Scraping

In [50]:
#Dependencies
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import requests
import pandas as pd
import time
import os

In [80]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======


[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [/Users/david/.wdm/drivers/chromedriver/mac64/89.0.4389.23/chromedriver] found in cache


### NASA Mars News

In [53]:
#Use splinter to browse through page
news_url="https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
browser.visit(news_url)

In [54]:
# HTML object
html = browser.html

# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')
# print(soup.prettify())

#Select latest category
browser.find_by_id('date').first.click()
browser.find_option_by_text('Latest').first.click()

#Select All categories
browser.find_by_id('categories').first.click()
browser.find_option_by_text('All Categories').first.click()
browser.find_by_id('categories').first.click()
time.sleep(0)

In [55]:
#Collect the latest News Title
results=soup.find('li', class_="slide")

# Pase the first title from the results 
news_title =results.find('div',class_='content_title').text.strip()

# Parse out the first paragraph associatd with the tile from results
news_p=results.find('div',class_='article_teaser_body').text.strip()

# #Print Title and Paragraph
# print(f'Title: {news_title}')
# print()
# print(f'Paragraph: {news_p}')

### JPL Mars Space Images - Featured Image

In [65]:
#Use splinter to browse through page
images_url="https://www.jpl.nasa.gov/images?search=&category=Mars"
browser.visit(images_url)
time.sleep(1)

In [66]:
# HTML object
html = browser.html

# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')
# print(soup.prettify())

In [69]:
browser.find_by_id('filter_Mars').click()
time.sleep(0.5)
featured_image_url=browser.find_by_css('.BaseImage')['data-src']

In [70]:
# print(featured_image_url)


### Mars Facts

In [72]:
#Use splinter to browse through page
facts_url="https://space-facts.com/mars/"
browser.visit(facts_url)

In [73]:
# Use Panda's `read_html` to parse the url
facts_tables = pd.read_html(facts_url)
# facts_tables

mars_0_df=facts_tables[0]
# mars_0_df

column_names={
    0: "Description",
    1: "Mars Fact"
}

mars_1_df=mars_0_df.rename(columns=column_names)
# mars_1_df

mars_df=mars_1_df.set_index("Description")
# mars_df

mars_facts_html=mars_df.to_html()

### Mars Hemispheres

In [81]:
#Use splinter to browse through page
hemispheres_url="https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemispheres_url)

In [76]:
# HTML object
html = browser.html

# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')
# print(soup.prettify())

In [82]:
# Retrieve all elements that contain hemisphere information
#Find the image link to access the high resolution image

results = soup.find_all('div', class_='item')
results

hemisphere_image_urls=[]
count=0

for each_result in results: 
    
    title=each_result.find('h3').text

    
    browser.find_by_tag("h3")[count].click()
    time.sleep(0.5)
    
    count=count+1
    
    html = browser.html

    img_url=browser.find_by_text('Sample')['href']
    
    hemisphere_dict={
        "title": title,
        "img_url":img_url
    }
    
    hemisphere_image_urls.append(hemisphere_dict)
    
    time.sleep(0.5)
    
    browser.visit(hemispheres_url)
    

In [83]:
# print(hemisphere_image_urls)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]


### Quit Browser

In [84]:
browser.quit()

## Step 2 - MongoDB and Flask Application

In [85]:
mars_dict={
    "Title":news_title,
    "Paragraph":news_p,
    "FeaturedImage": featured_image_url,
    "MarsFacts": mars_facts_html,
    "HemisphereImages": hemisphere_image_urls
}

print(mars_dict)

{'Title': 'Another First: Perseverance Captures the Sounds of Driving on Mars', 'Paragraph': 'NASA’s newest rover recorded audio of itself crunching over the surface of the Red Planet, adding a whole new dimension to Mars exploration.', 'FeaturedImage': 'https://d2pn8kiwq2w21t.cloudfront.net/images/jpegPIA24505.2e16d0ba.fill-400x400-c50.jpg', 'MarsFacts': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars Fact</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</